## unit fisher
beta = [0,2]    alpha = 0.333
init_beta = 1
beta_mean = 0.91

without fisher :
    acc : 95.48---93.47
    drop : 2.01
with fisher :
    acc : 95.48---58.23
    drop : 37.24

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import sys
import numpy as np
import pickle

sys.path.append('D:\\Program\\MyCode\\Round_robin_SL\\Round-Robin')
from models import *
from clients_datasets import *
from tqdm.notebook import tqdm
from utils import *
from AttFunc import *
from Fisher_LeNet import *

In [2]:
batch_size = 600
epochs = 30
NC = 10
dataset = 'svhn'

clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
clients_testloader = load_clients_testsets(dataset, NC, batch_size)

server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
client_level = 1
server_level = 6

criterion = torch.nn.CrossEntropyLoss()

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [3]:
for name, param in server.named_parameters():
    print(f"Parameter name: {name}, Parameter size: {param.size()}")

Parameter name: conv1.0.weight, Parameter size: torch.Size([64, 3, 3, 3])
Parameter name: conv1.0.bias, Parameter size: torch.Size([64])
Parameter name: conv1.1.weight, Parameter size: torch.Size([64])
Parameter name: conv1.1.bias, Parameter size: torch.Size([64])
Parameter name: conv2.0.weight, Parameter size: torch.Size([128, 64, 3, 3])
Parameter name: conv2.0.bias, Parameter size: torch.Size([128])
Parameter name: conv2.1.weight, Parameter size: torch.Size([128])
Parameter name: conv2.1.bias, Parameter size: torch.Size([128])
Parameter name: res1.0.weight, Parameter size: torch.Size([128, 128, 3, 3])
Parameter name: res1.0.bias, Parameter size: torch.Size([128])
Parameter name: res1.1.weight, Parameter size: torch.Size([128])
Parameter name: res1.1.bias, Parameter size: torch.Size([128])
Parameter name: res1.3.weight, Parameter size: torch.Size([128, 128, 3, 3])
Parameter name: res1.3.bias, Parameter size: torch.Size([128])
Parameter name: res1.4.weight, Parameter size: torch.Size([

In [3]:
clients_acc0 = []
#################################################################################
###########################   Normal training   #################################
# train
server.train()
for i in range(NC):
    clients[i].train()
server.apply(init_weights)
clients[0].apply(init_weights)
last_trained_params = clients[0].state_dict()
for epoch in tqdm(range(epochs), desc="Normal Training", unit="eopch"):
# for epoch in range(epochs):
    for idx, client in enumerate(clients):
        client.load_state_dict(last_trained_params)
        for j, data in enumerate(clients_trainloader[idx]):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            clients_opts[idx].zero_grad()
            server_opt.zero_grad()
            loss = criterion(output, labels)
            loss.backward()
            clients_opts[idx].step()
            server_opt.step()
        last_trained_params = client.state_dict()
for i in range(NC):
    clients[i].load_state_dict(last_trained_params)
# test
server.eval()
for i in range(NC):
    clients[i].eval()
with torch.no_grad():
    for idx, client in enumerate(tqdm(clients)):
        correct = 0
        total = 0
        acc0 = 0
        for data in clients_testloader[idx]:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()

            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            _, pre = torch.max(output.data, 1)
            total += images.shape[0]
            correct += (pre == labels).sum().item()
        acc0 = 100 * correct / total
        clients_acc0.append(acc0)
acc0 = np.mean(clients_acc0)
print(acc0)

Normal Training:   0%|          | 0/30 [00:00<?, ?eopch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

95.47863064772834


In [4]:
am = ResNet_9().cuda()
am_train(am, clients_trainloader[8], dataset)
fisher_matrix = {}
for param_name, param in am.named_parameters():
    if param_name == 'conv1.0.weight':
        grad = param.grad.cpu().detach().numpy()
        if param_name not in fisher_matrix:
            fisher_matrix[param_name] = grad ** 2
        else:
            fisher_matrix[param_name] += grad ** 2
    if param_name == 'conv1.0.bias':
        grad = param.grad.cpu().detach().numpy()
        if param_name not in fisher_matrix:
            fisher_matrix[param_name] = grad ** 2
        else:
            fisher_matrix[param_name] += grad ** 2

Training attack model:   0%|          | 0/100 [00:00<?, ?eopch/s]

In [5]:
am_params = torch.load('../Round_robin_SL/Round-Robin/params/am_svhn_params.pth')
am_acc0 = am_test(am, am_params, clients_testloader[8], 0)
print(-am_acc0)

90.62620053784096


In [6]:
att_type = 'unit'
acc0 = 95.48
betas = []

for i in tqdm(range(10), desc="Finding beta", unit="iter"):
    beta = simulated_annealing(dataset, 1, 500, att_type, acc0,  clients_testloader[8])
    betas.append(beta)
    print(beta)

Finding beta:   0%|          | 0/10 [00:00<?, ?iter/s]

0.5402048645906912
0.9147156960660681
1.4169588898217909
0.8116515774045837
1.0495300645230783
0.7150690074701289
0.8816934881672096
1.605457927265741
0.5480967928897891
0.4930106289167863


In [7]:
beta_mean, beta_range = data_process(betas)
print(beta_mean)

0.9143977065402349


---------------------------------------------------------
## without fisher

In [8]:
acc0 = 95.48
att_type = 'unit'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'svhn'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 6

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]
                mal_params = perturbation(beta=beta, bp=benign_params, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [9]:
print(acc1_varying)
print(drop_varying)

[95.02149829483055, 93.30244539415409, 93.53357198033936, 93.57710632243429, 86.23616965654071]
[0.4585017051694507, 2.177554605845912, 1.9464280196606463, 1.9028936775657104, 9.243830343459312]


In [10]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
93.47104123230925
0.2746609282801984
---------------------
drop:
2.0089587676907565
0.2746609282802015


--------------------------------------------------------
## with fisher

In [12]:
weight_positions = []
bias_positions = []
weight_positions.append(find_positions(fisher_matrix['conv1.0.weight'], 0.333))
bias_positions.append(find_positions(fisher_matrix['conv1.0.bias'], 0.333))

In [13]:
acc0 = 95.48
att_type = 'unit'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'svhn'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 6

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]
                mal_params = fisher_perturbation(client_level, beta, benign_params, weight_positions, bias_positions, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [15]:
print(acc1_varying)
print(drop_varying)

[7.655968860224496, 51.065995722765834, 65.41051189704108, 72.8689652964528, 77.18730750823373]
[87.82403113977551, 44.41400427723418, 30.069488102958918, 22.611034703547194, 18.29269249176627]


In [16]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
58.238253809903455
14.344516174275242
---------------------
drop:
37.24174619009655
14.34451617427526
